1. Mount Google Drive environment

In [1]:
# Mount Google Drive
# Re-run this cell every time you open this notebook
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2. Look at text data

In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

pd.set_option('display.max_colwidth', None)   # Don't truncate the data when printed

In [ ]:
path = '/content/drive/MyDrive/CS5344 Project/Data and Codes/'
train_df = pd.read_csv(os.path.join(path, "train_embedding.csv"))
dev_df = pd.read_csv(os.path.join(path, "dev_embedding.csv"))
test_df = pd.read_csv(os.path.join(path, "test_embedding.csv"))

In [ ]:
display(train_df.info())
display(dev_df.info())
display(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9989 entries, 0 to 9988
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Sr No.                  9989 non-null   int64 
 1   Utterance               9989 non-null   object
 2   Speaker                 9989 non-null   object
 3   Emotion                 9989 non-null   object
 4   Sentiment               9989 non-null   object
 5   Dialogue_ID             9989 non-null   int64 
 6   Utterance_ID            9989 non-null   int64 
 7   Season                  9989 non-null   int64 
 8   Episode                 9989 non-null   int64 
 9   StartTime               9989 non-null   object
 10  EndTime                 9989 non-null   object
 11  cleaned_text            9989 non-null   object
 12  cleaned_text_new        9989 non-null   object
 13  word2vec_embedding      9989 non-null   object
 14  word2vec_embedding_new  9989 non-null   object
 15  bert

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1109 entries, 0 to 1108
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Sr No.                  1109 non-null   int64 
 1   Utterance               1109 non-null   object
 2   Speaker                 1109 non-null   object
 3   Emotion                 1109 non-null   object
 4   Sentiment               1109 non-null   object
 5   Dialogue_ID             1109 non-null   int64 
 6   Utterance_ID            1109 non-null   int64 
 7   Season                  1109 non-null   int64 
 8   Episode                 1109 non-null   int64 
 9   StartTime               1109 non-null   object
 10  EndTime                 1109 non-null   object
 11  cleaned_text            1109 non-null   object
 12  cleaned_text_new        1109 non-null   object
 13  word2vec_embedding      1109 non-null   object
 14  word2vec_embedding_new  1109 non-null   object
 15  bert

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2610 entries, 0 to 2609
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Sr No.                  2610 non-null   int64 
 1   Utterance               2610 non-null   object
 2   Speaker                 2610 non-null   object
 3   Emotion                 2610 non-null   object
 4   Sentiment               2610 non-null   object
 5   Dialogue_ID             2610 non-null   int64 
 6   Utterance_ID            2610 non-null   int64 
 7   Season                  2610 non-null   int64 
 8   Episode                 2610 non-null   int64 
 9   StartTime               2610 non-null   object
 10  EndTime                 2610 non-null   object
 11  cleaned_text            2610 non-null   object
 12  cleaned_text_new        2610 non-null   object
 13  word2vec_embedding      2610 non-null   object
 14  word2vec_embedding_new  2610 non-null   object
 15  bert

None

In [ ]:
# Preview 1 record
train_df.iloc[0]

,0
Sr No.,1
Utterance,also I was the point person on my companys transition from the KL-5 to GR-6 system.
Speaker,Chandler
Emotion,neutral
Sentiment,neutral
Dialogue_ID,0
Utterance_ID,0
Season,8
Episode,21
StartTime,"00:16:16,059"


3. Look at extracted face images

In [3]:
# Check no. of extracted frames (file counts)
!find "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/train_frames_speaker" -type f | wc -l
!find "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/dev_frames_speaker" -type f | wc -l
!find "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/test_frames_speaker" -type f | wc -l

8871
962
2357


In [ ]:
# Check no. of extracted facial images for largest_face_extraction (file counts)
!find "/content/drive/MyDrive/CS5344 Project/Data and Codes/largest_face_extraction/train_face/" -type f | wc -l
!find "/content/drive/MyDrive/CS5344 Project/Data and Codes/largest_face_extraction/dev_face/" -type f | wc -l
!find "/content/drive/MyDrive/CS5344 Project/Data and Codes/largest_face_extraction/test_face/" -type f | wc -l

9780
1088
2545


**Follow-up required: To extract 1 face from each frame.**

**Below codes are ran on clustered face**


4a. To resize extracted faces, and extract features (train set).


In [5]:
!pip install facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.

In [3]:
from facenet_pytorch import InceptionResnetV1
import torch

# Load FaceNet (InceptionResnetV1) pretrained on VGGFace2
facenet = InceptionResnetV1(pretrained='vggface2').eval().to('cuda' if torch.cuda.is_available() else 'cpu')

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [4]:
from torchvision import transforms

# Define FaceNet preprocessing
preprocess = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)  # Normalize to [-1, 1] range
])

def get_facenet_embedding(image_path):
    img = Image.open(image_path).convert('RGB')
    img_tensor = preprocess(img).unsqueeze(0)  # Shape: (1, 3, 160, 160)
    img_tensor = img_tensor.to('cuda' if torch.cuda.is_available() else 'cpu')

    with torch.no_grad():
        embedding = facenet(img_tensor)  # Shape: (1, 512)

    return embedding.cpu().numpy()[0]  # Return as 1D numpy array

In [ ]:
input_dir = "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/train_frames_speaker"
output_dir = "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/train_frames_speaker_embeddings"
os.makedirs(output_dir, exist_ok=True)

image_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.jpg')])

for filename in tqdm(image_files, desc="Extracting FaceNet embeddings"):
    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, filename.replace(".jpg", ".npy"))

    if os.path.exists(output_path):
        continue  # Skip if already done

    try:
        embedding = get_facenet_embedding(input_path)
        np.save(output_path, embedding)
    except Exception as e:
        print(f"\nError processing {filename}: {e}")

Extracting FaceNet embeddings: 100%|██████████| 8871/8871 [21:44<00:00,  6.80it/s]



4b. To resize extracted faces, and extract features (dev set).


In [ ]:
input_dir = "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/dev_frames_speaker"
output_dir = "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/dev_frames_speaker_embeddings"
os.makedirs(output_dir, exist_ok=True)

image_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.jpg')])

for filename in tqdm(image_files, desc="Extracting FaceNet embeddings"):
    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, filename.replace(".jpg", ".npy"))

    if os.path.exists(output_path):
        continue  # Skip if already done

    try:
        embedding = get_facenet_embedding(input_path)
        np.save(output_path, embedding)
    except Exception as e:
        print(f"\nError processing {filename}: {e}")

Extracting FaceNet embeddings: 100%|██████████| 962/962 [02:28<00:00,  6.47it/s]



4c. To resize extracted faces, and extract features (test set).


In [5]:
input_dir = "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/test_frames_speaker"
output_dir = "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/test_frames_speaker_embeddings"
os.makedirs(output_dir, exist_ok=True)

image_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.jpg')])

for filename in tqdm(image_files, desc="Extracting FaceNet embeddings"):
    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, filename.replace(".jpg", ".npy"))

    if os.path.exists(output_path):
        continue  # Skip if already done

    try:
        embedding = get_facenet_embedding(input_path)
        np.save(output_path, embedding)
    except Exception as e:
        print(f"\nError processing {filename}: {e}")

Extracting FaceNet embeddings: 100%|██████████| 2357/2357 [05:09<00:00,  7.61it/s]


In [6]:
# Check no. of embedding files
!find "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/train_frames_speaker_embeddings" -type f | wc -l
!find "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/dev_frames_speaker_embeddings" -type f | wc -l
!find "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/test_frames_speaker_embeddings" -type f | wc -l

8871
962
2357


5. Load all face embeddings

In [8]:
# Paths to embedding directories
train_dir = "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/train_frames_speaker_embeddings"
dev_dir   = "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/dev_frames_speaker_embeddings"
test_dir  = "/content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/test_frames_speaker_embeddings"

def load_embeddings(embedding_dir):
    files = sorted([f for f in os.listdir(embedding_dir) if f.endswith(".npy")])
    embeddings = []
    filenames = []

    # Use tqdm to show progress for the files being processed
    for file in tqdm(files, desc=f"Loading embeddings from {embedding_dir}", unit="file"):
        path = os.path.join(embedding_dir, file)
        if os.path.getsize(path) > 0:
            try:
                embedding = np.load(path)
                embeddings.append(embedding)
                filenames.append(file.replace(".npy", ""))
            except Exception as e:
                print(f"Error loading {file}: {e}")
        else:
            print(f"Skipping empty file: {file}")

    return np.stack(embeddings), filenames

# Load all three sets with progress bars
train_embedding_matrix, train_filenames = load_embeddings(train_dir)
dev_embedding_matrix, dev_filenames     = load_embeddings(dev_dir)
test_embedding_matrix, test_filenames   = load_embeddings(test_dir)

# Check shapes
print("Train embedding shape:", train_embedding_matrix.shape)
print("Dev embedding shape:", dev_embedding_matrix.shape)
print("Test embedding shape:", test_embedding_matrix.shape)

Loading embeddings from /content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/train_frames_speaker_embeddings: 100%|██████████| 8871/8871 [02:48<00:00, 52.70file/s] 
Loading embeddings from /content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/dev_frames_speaker_embeddings: 100%|██████████| 962/962 [00:16<00:00, 59.65file/s] 
Loading embeddings from /content/drive/MyDrive/CS5344 Project/Data and Codes/frames_speaker/test_frames_speaker_embeddings: 100%|██████████| 2357/2357 [00:30<00:00, 78.27file/s] 


Train embedding shape: (8871, 512)
Dev embedding shape: (962, 512)
Test embedding shape: (2357, 512)


In [9]:
# Preview a face embedding
print(train_embedding_matrix[0].shape)
train_embedding_matrix[0]

(512,)


array([ 3.52571011e-02, -1.16681366e-03, -8.03913698e-02, -2.60037836e-02,
        2.93896198e-02, -5.56406230e-02, -2.63683754e-03,  8.99192989e-02,
        4.61612940e-02,  7.12634325e-02,  1.44226914e-02, -6.62601367e-03,
        2.76612248e-02, -3.67297307e-02,  7.67826382e-03,  4.43019792e-02,
       -3.01454999e-02, -9.92868841e-02, -1.41271316e-02,  8.00280832e-03,
       -5.20792417e-02,  2.09805742e-02, -3.65079306e-02, -1.07695488e-02,
        1.97831262e-02,  8.53650458e-03,  1.15803368e-02, -6.81540966e-02,
        1.59860663e-02,  9.81888850e-04, -1.63855739e-02, -1.84671581e-02,
        1.56686474e-02,  4.15405221e-02, -4.81996015e-02,  2.85862829e-04,
        5.73319905e-02,  2.61600856e-02,  2.16038749e-02, -6.54685423e-02,
       -3.97270769e-02, -2.47791759e-03,  6.41441941e-02,  3.32436636e-02,
       -8.33166316e-02,  4.72144932e-02,  5.61041478e-03, -3.61079611e-02,
        7.81312771e-03,  7.83310086e-03,  8.65176693e-02, -9.15803853e-03,
       -2.27511656e-02, -